In [1]:
import tensorflow as tf
import numpy as np

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
# Input
N = 5
a = tf.constant(np.arange(1,N**2+1).reshape(N,N), tf.float32)
#[[0. 1. 2.]
# [3. 4. 5.]
# [6. 7. 8.]]

b = tf.zeros_like(a)
print((type(b)))
c = tf.reshape(tf.stack([a,b], 2),
               [-1, tf.shape(a)[1]+tf.shape(b)[1]])[:,:-1]

with tf.Session() as sess:
   print(sess.run(c))
#[[0. 0. 1. 0. 2.]
# [3. 0. 4. 0. 5.]
# [6. 0. 7. 0. 8.]]

<class 'tensorflow.python.framework.ops.Tensor'>
[[ 1.  0.  2.  0.  3.  0.  4.  0.  5.]
 [ 6.  0.  7.  0.  8.  0.  9.  0. 10.]
 [11.  0. 12.  0. 13.  0. 14.  0. 15.]
 [16.  0. 17.  0. 18.  0. 19.  0. 20.]
 [21.  0. 22.  0. 23.  0. 24.  0. 25.]]


In [4]:
def explode_width(tensor, num_zeros = 2):
    h_zero = tf.zeros_like(tensor)
    h_expand = tf.reshape(tf.stack([tensor,h_zero, h_zero], 2),[-1, tf.shape(tensor)[1]*3])[:,:-2]
    return h_expand

def explode_tensor(tensor, num_zeros = 2):
    h_expand = explode_width(tensor, num_zeros=num_zeros)
    rot = tf.transpose(h_expand)
    rot1 = explode_width(rot, num_zeros=num_zeros)
    return rot1

class explode(keras.layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        self.num_zeros = kwargs.pop('num_zeros')
        super(explode, self).__init__(**kwargs)

    def build(self, input_shape):
        pass
#         shape = tf.TensorShape((input_shape[1], self.output_dim))
#         # Create a trainable weight variable for this layer.
#         self.kernel = self.add_weight(name='kernel',
#                                       shape=shape,
#                                       initializer='uniform',
#                                       trainable=True)
#         # Be sure to call this at the end
#         super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        return explode_width(inputs, num_zeros = 2)

    def compute_output_shape(self, input_shape):
        shape = tf.multiply(tf.TensorShape(input_shape), tf.convert_to_tensor(self.num_zeros) - 1
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

SyntaxError: invalid syntax (<ipython-input-4-8dc4953c6b56>, line 35)

In [5]:
a = tf.constant(np.arange(1,N**2+1).reshape(N,N), tf.float32)
with tf.Session() as sess:
    b = sess.run(explode_tensor(a))
    print(b)
    print(type(b))

NameError: name 'explode_tensor' is not defined

In [10]:
def combine(loc, per, filter_size = 3):
    #Size of incoming image
    loc_x,loc_y = loc.shape
    per_x,per_y = per.shape
    n = loc_x
    square_radius = (n//2)*filter_size  
    assert(loc_x == loc_y == per_x == per_y)
    #How big do we need to expand the peripherial image by?
    expand_dist = 2*filter_size+2
    #expand_dist = n + 1
    print(expand_dist)
    #How big does the new matrix have to be?
    big_x = big_y = 3*n + 4*filter_size + 2
    center_x = center_y = (big_x/2)
    assert(big_x//2 == big_x/2)
    big_array = np.zeros([big_x,big_y])
    for i in range(loc_x):
        for j in range(loc_y):
            temp_val = per[i,j]
            big_i = filter_size * i + expand_dist
            big_j = filter_size * j + expand_dist
            for i_change in [-expand_dist, 0, expand_dist]:
                for j_change in [-expand_dist, 0, expand_dist]:
                    big_array[big_i+i_change,big_j+j_change] = temp_val
                     
    return big_array[center_x-square_radius:center_x+1+square_radius,
                    center_y-square_radius:center_y+1+square_radius]

In [12]:
N = 11
array_test = np.arange(1,N**2 + 1).reshape(N,N)
print(array_test)
big_array_test = combine(array_test,array_test)
print(big_array_test)
print(big_array_test.shape)

[[  1   2   3   4   5   6   7   8   9  10  11]
 [ 12  13  14  15  16  17  18  19  20  21  22]
 [ 23  24  25  26  27  28  29  30  31  32  33]
 [ 34  35  36  37  38  39  40  41  42  43  44]
 [ 45  46  47  48  49  50  51  52  53  54  55]
 [ 56  57  58  59  60  61  62  63  64  65  66]
 [ 67  68  69  70  71  72  73  74  75  76  77]
 [ 78  79  80  81  82  83  84  85  86  87  88]
 [ 89  90  91  92  93  94  95  96  97  98  99]
 [100 101 102 103 104 105 106 107 108 109 110]
 [111 112 113 114 115 116 117 118 119 120 121]]
8
[[  1.   4.   0.   2.   5.   0.   3.   6.   1.   4.   7.   2.   5.   8.
    3.   6.   9.   4.   7.  10.   5.   8.  11.   6.   9.   0.   7.  10.
    0.   8.  11.]
 [ 34.  37.   0.  35.  38.   0.  36.  39.  34.  37.  40.  35.  38.  41.
   36.  39.  42.  37.  40.  43.  38.  41.  44.  39.  42.   0.  40.  43.
    0.  41.  44.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0. 